In [1]:
!python --version

Python 3.10.12


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.5/719.5 kB 10.0 MB/s eta 0:00:00


In [3]:
import ultralytics
ultralytics.__version__

'8.1.24'

In [4]:
!pip install torch

In [5]:
import torch
torch.__version__

'2.1.0+cu121'

In [6]:
torch.cuda.get_device_name(0)

'Tesla T4'

# Detect, track and count Persons

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
# %cd yolov8_DeepSORT

In [9]:
from ultralytics import YOLO

import time
import torch
import cv2
import torch.backends.cudnn as cudnn
from PIL import Image
import colorsys
import numpy as np

100%|██████████| 6.23M/6.23M [00:00<00:00, 114MB/s]



image 1/1 /content/drive/MyDrive/Major Project/Images/Jumping/IMG_20231027_180529681_BURST0011.jpg: 640x480 1 person, 5 potted plants, 84.6ms
Speed: 11.8ms preprocess, 84.6ms inference, 993.3ms postprocess per image at shape (1, 3, 640, 480)
Results saved to runs/detect/predict
[58.0, 0.0, 58.0, 58.0, 58.0, 58.0]
Class: potted plant
Class: person
Class: potted plant
Class: potted plant
Class: potted plant
Class: potted plant


# DeepSORT

In [10]:
%cd /content/drive/MyDrive/Tracking-and-counting-Using-YOLOv8-and-DeepSORT/

/content/drive/MyDrive/Tracking-and-counting-Using-YOLOv8-and-DeepSORT


In [11]:
import cv2
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort
from deep_sort.sort.tracker import Tracker

deep_sort_weights = 'deep_sort/deep/checkpoint/ckpt.t7'
tracker = DeepSort(model_path=deep_sort_weights, max_age=120)

In [29]:
# Define the video path
video_path = '/content/drive/MyDrive/Tracking-and-counting-Using-YOLOv8-and-DeepSORT/input_video.mp4'

cap = cv2.VideoCapture(video_path)

# Get the video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
output_path = '/content/drive/MyDrive/Tracking-and-counting-Using-YOLOv8-and-DeepSORT/testnewopyolo.mp4'
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [30]:
frames = []

unique_track_ids = set()

# Main Code

In [31]:
# Placeholder function for extracting appearance features
def extract_appearance_features(frame, bounding_box):
    # Ensure the bounding box is valid before extracting features
    if bounding_box.size > 0:  # Check if the bounding box is not empty
        # Example: Extract color histogram as appearance feature
        # Convert the bounding box region to HSV color space
        hsv = cv2.cvtColor(bounding_box, cv2.COLOR_BGR2HSV)

        # Calculate the color histogram
        hist_hue = cv2.calcHist([hsv], [0], None, [180], [0, 180])
        hist_saturation = cv2.calcHist([hsv], [1], None, [256], [0, 256])
        hist_value = cv2.calcHist([hsv], [2], None, [256], [0, 256])

        # Normalize the histograms
        cv2.normalize(hist_hue, hist_hue, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        cv2.normalize(hist_saturation, hist_saturation, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)
        cv2.normalize(hist_value, hist_value, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX)

        # Concatenate the histograms as the appearance feature
        appearance_feature = np.concatenate((hist_hue.flatten(), hist_saturation.flatten(), hist_value.flatten()))

        return appearance_feature
    else:
        return None  # Return None if the bounding box is empty or invalid

# Placeholder function for matching appearances
def match_appearances(current_appearance, past_appearance):
    # Implement appearance matching logic here

    # Iterate through appearance features in the current and past appearances
    for feature_id, current_feature in current_appearance.items():
        # Check if the feature_id exists in the past appearance dictionary
        if feature_id in past_appearance:
            # Calculate the Euclidean distance between current and past appearance features
            distance = np.linalg.norm(current_feature - past_appearance[feature_id])

            # Set a threshold for similarity/dissimilarity
            threshold = 0.5  # Adjust this threshold based on your feature representation

            # If the distance is below the threshold, consider it a match
            if distance < threshold:
                return feature_id  # Return the matched feature ID

    # If no match is found, return None
    return None
# Initialize a dictionary to store appearance information for past tracks
i = 0
counter, fps, elapsed = 0, 0, 0
start_time = time.perf_counter()
appearance_memory = {}

while cap.isOpened():
    ret, frame = cap.read()

    if ret:
        og_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = og_frame.copy()

        model = YOLO("/content/yolov8n.pt")  # load a pretrained model (recommended for training)

        results = model(frame, device=0, classes=0, conf=0.8)

        class_names = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

        for result in results:
            boxes = result.boxes  # Boxes object for bbox outputs
            probs = result.probs  # Class probabilities for classification outputs
            cls = boxes.cls.tolist()  # Convert tensor to list
            xyxy = boxes.xyxy
            conf = boxes.conf
            xywh = boxes.xywh  # box with xywh format, (N, 4)
            print("Cls value:", cls)
            for class_index in cls:
                class_name = class_names[int(class_index)]
                print("Class:", class_name)

        pred_cls = np.array(cls)
        conf = conf.detach().cpu().numpy()
        xyxy = xyxy.detach().cpu().numpy()
        bboxes_xywh = xywh
        bboxes_xywh = xywh.cpu().numpy()
        bboxes_xywh = np.array(bboxes_xywh, dtype=float)

        tracks = tracker.update(bboxes_xywh, conf, og_frame)

        # Initialize a dictionary to store current detections' appearances
        current_appearances = {}

        for track in tracker.tracker.tracks:
            track_id = track.track_id
            # Get bounding box information
            x1, y1, x2, y2 = track.to_tlbr()
            bounding_box = frame[int(y1):int(y2), int(x1):int(x2)]
            # Extract appearance features for the current track
            appearance = extract_appearance_features(frame, bounding_box)

            # Check if track ID exists in appearance memory
            if track_id in appearance_memory:
                # Compare current appearance with past appearances
                matched_id = match_appearances(current_appearances, appearance_memory[track_id])

                if matched_id is not None:
                    # Update the track ID if matched with past appearance
                    track.track_id = matched_id

            # Update the appearance memory with the current appearance
            appearance_memory[track_id] = appearance

            # Draw bounding box and label with updated track ID
            for track in tracker.tracker.tracks:
              track_id = track.track_id
              x1, y1, x2, y2 = track.to_tlbr()

              # Set color values for different tracks
              color = (255, 0, 0)  # Default color

              # Update color based on track ID
              if track_id in appearance_memory:
                  color = (0, 255, 0)  # Set a different color if the appearance is matched

              # Draw bounding box
              cv2.rectangle(og_frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

              # Create label with updated track ID
              label = f"Track ID: {track_id}"

              # Display the label near the top-left corner of the bounding box
              cv2.putText(og_frame, label, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        # Other parts of the code: Counting, FPS calculation, writing frames, etc.
        person_count = len(tracks)

        # Update FPS and display on frame (if needed)
        # Assuming fps is calculated elsewhere, display FPS on the top-left corner of the frame
        cv2.putText(og_frame, f"FPS: {fps:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        # Writing the frame to the output video file
        out.write(cv2.cvtColor(og_frame, cv2.COLOR_RGB2BGR))

    else:
        break
cap.release()
out.release()
cv2.destroyAllWindows()


0: 480x640 1 person, 11.4ms
Speed: 1.7ms preprocess, 11.4ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Cls value: [0.0]
Class: person

0: 480x640 1 person, 7.2ms
Speed: 1.3ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Cls value: [0.0]
Class: person

0: 480x640 1 person, 7.2ms
Speed: 1.3ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Cls value: [0.0]
Class: person

0: 480x640 1 person, 7.7ms
Speed: 1.2ms preprocess, 7.7ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Cls value: [0.0]
Class: person

0: 480x640 1 person, 7.0ms
Speed: 1.3ms preprocess, 7.0ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Cls value: [0.0]
Class: person

0: 480x640 1 person, 7.4ms
Speed: 2.5ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 480, 640)
Cls value: [0.0]
Class: person

0: 480x640 1 person, 7.0ms
Speed: 1.3ms preprocess, 7.0ms inference

In [ ]:
# torch.save(model.state_dict(), "model.pt")

In [ ]:
# model = torch.load("model.pt")